#### Question 1. 
How many employees spent above their given stipend? Extract the name of the employee, the assigned stipend, and the actual amount spent.

In [19]:
# fetching the name and stipend from the bissan database
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host = 'localhost',
    user = 'admin',
    dbname = 'bissan',
    password = 'admin',
    port =9355
)

cur = conn.cursor()

In [20]:
query = """
    SELECT 
	DISTINCT name, d.entitled_bonus AS stipend
FROM employees e
LEFT JOIN departments d
	ON e.department_id = d._id;
"""

cur.execute(query)
res = cur.fetchall()

In [21]:
cur.description

(Column(name='name', type_code=1043), Column(name='stipend', type_code=23))

In [22]:
res

[('Robert Green', 6832),
 ('James Sawyer', 6832),
 ('Dieter Klinger', 7377),
 ('Chase Fabrizio', 6913),
 ('Scott Davis', 6913),
 ('Calvin Jones', 6086),
 ('Rashid Abramov', 7377),
 ('Adam P. Jones', 6832),
 ('Gerry Mcfarland', 6086),
 ('John Bentley', 6913),
 ('Alysoun Mclaughlin', 6913),
 ('Thomas Jordan', 7377),
 ('Thomas Manger', 7377),
 ('Joshua Hart', 7377),
 ('John Bentley', 6086),
 ('Paula Carney', 7377),
 ('Michael Toney', 6086),
 ('Darryl Mcswain', 6913),
 ('Jim Lynch', 6913),
 ('Mark Blackburn', 6832),
 ('John Fitzgerald', 6832),
 ('Gregory Lawrence', 6913),
 ('Scott Webster', 6832),
 ('Mackall Heflin', 6086),
 ('Gary Walker', 6832),
 ('Ben Kaufman', 6086),
 ('Laura Chief', 6086),
 ('Ashley Robinson', 6832),
 ('Timothy Burns', 6832),
 ('Alan Butsch', 6913),
 ('Michael Donohue', 6086),
 ('Jeffery Ewart', 6832),
 ('Nancy Moseley', 7377),
 ('Marjorie Williams', 6913),
 ('Heather Johnson', 6086),
 ('Mark Poole', 7377),
 ('Atiquilla Panjshiri', 6832),
 ('Patrick Hennessy', 6832),


In [23]:
# storing db extracts/result into dataframe
employees_df = pd.DataFrame(res, columns= ['Name', 'Stipend'], index= None)
employees_df

,Name,Stipend
0,Robert Green,6832
1,James Sawyer,6832
2,Dieter Klinger,7377
3,Chase Fabrizio,6913
4,Scott Davis,6913
...,...,...
408,Lisa Killen,6086
409,Jason Giza,6832
410,Dana Stroman,7377
411,Joseph Webster,6832


In [24]:
# checking duplicates in the whole dataframe
employees_df[employees_df.duplicated()]

,Name,Stipend


In [25]:
employees_df = employees_df.drop_duplicates(subset= ['Name']) #dropping duplicates in the name column
employees_df

,Name,Stipend
0,Robert Green,6832
1,James Sawyer,6832
2,Dieter Klinger,7377
3,Chase Fabrizio,6913
4,Scott Davis,6913
...,...,...
408,Lisa Killen,6086
409,Jason Giza,6832
410,Dana Stroman,7377
411,Joseph Webster,6832


In [26]:
travels_data = pd.read_csv("cleaned_travel_dataset.csv") #changing the cleaned dataset into dataframe
travels_data.head(3)

,Unnamed: 0,Employee,first_name,last_name,state,city,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel
0,0,Aaron Salter,Aaron,Salter,MD,OCEAN CITY,2017-05-31,2017-06-02,644.10,Conference
1,1,Abigail Ratnofsky,Abigail,Ratnofsky,CA,SAN DIEGO,2016-01-24,2016-01-29,2962.70,Conference
2,2,Adam Kisthardt,Adam,Kisthardt,FL,ORLANDO,2018-10-04,2018-10-09,1891.19,Conference


In [27]:
travels_data.drop('Unnamed: 0', axis=1, inplace=True)

In [28]:
employees_expense = travels_data[['Employee', 'Actual Total Expenses']] #extracting the needed data

In [29]:
employees_expense[employees_expense.duplicated()]

,Employee,Actual Total Expenses


In [30]:
employees_expense = employees_expense.groupby(['Employee']).sum() # get the total sum for each employee
employees_expense

,Actual Total Expenses
Employee,
Aaron Salter,644.10
Abigail Ratnofsky,2962.70
Adam Kisthardt,1891.19
Adam P. Jones,324.36
Alan Butsch,2141.51
...,...
Warren Jensen,502.45
Whitney Kujawa,567.00
William Kinna,2163.67


In [31]:
employees_expense = employees_expense.reset_index() #reset index to pandas' default
employees_expense

,Employee,Actual Total Expenses
0,Aaron Salter,644.10
1,Abigail Ratnofsky,2962.70
2,Adam Kisthardt,1891.19
3,Adam P. Jones,324.36
4,Alan Butsch,2141.51
...,...,...
396,Warren Jensen,502.45
397,Whitney Kujawa,567.00
398,William Kinna,2163.67
399,Wins Hinde,352.00


In [32]:
employees_df

,Name,Stipend
0,Robert Green,6832
1,James Sawyer,6832
2,Dieter Klinger,7377
3,Chase Fabrizio,6913
4,Scott Davis,6913
...,...,...
408,Lisa Killen,6086
409,Jason Giza,6832
410,Dana Stroman,7377
411,Joseph Webster,6832


In [33]:
# joining the two dataframes
employee_spending = employees_expense.set_index('Employee').join(employees_df.set_index('Name')).reset_index()
employee_spending

,Employee,Actual Total Expenses,Stipend
0,Aaron Salter,644.10,7377
1,Abigail Ratnofsky,2962.70,6086
2,Adam Kisthardt,1891.19,7377
3,Adam P. Jones,324.36,6832
4,Alan Butsch,2141.51,6913
...,...,...,...
396,Warren Jensen,502.45,6086
397,Whitney Kujawa,567.00,6832
398,William Kinna,2163.67,7377
399,Wins Hinde,352.00,6086


In [34]:
employees_expense.merge(employees_df, how= 'inner', left_on= 'Employee', right_on= 'Name')

,Employee,Actual Total Expenses,Name,Stipend
0,Aaron Salter,644.10,Aaron Salter,7377
1,Abigail Ratnofsky,2962.70,Abigail Ratnofsky,6086
2,Adam Kisthardt,1891.19,Adam Kisthardt,7377
3,Adam P. Jones,324.36,Adam P. Jones,6832
4,Alan Butsch,2141.51,Alan Butsch,6913
...,...,...,...,...
396,Warren Jensen,502.45,Warren Jensen,6086
397,Whitney Kujawa,567.00,Whitney Kujawa,6832
398,William Kinna,2163.67,William Kinna,7377
399,Wins Hinde,352.00,Wins Hinde,6086


In [35]:
employee_spending.to_csv('employee_spending.csv')